#  <font color='blue'> Pytorch to Libtorch</font>

In the previous section, we saw how Pytorch allows us to create an Intermediate-Representation of a network, with an idea of loading and inferring this `IR` over a different language.   
And this is where `Libtorch` comes up.  

In fact, converting the model to an `IR` is just a few lines of code.  
And it involves almost the stuff we discussed in the previous notebook.

Let's take a look as to how we can do this. 

First, we create a simple model `LeNet` in Pytorch and pass it inside the `torch.jit.script()` to get the `IR`. 
Once we get this `IR` of `LeNet`, we can save it as a `.pt` file so that it can be loaded within `Libtorch`.  
Then, we just need to load the file using `torch::jit::load("*.pt")`.


One thing to note is that in PyTorch, we used to have the model saved with only the `model-parameters ie .pth`, and we used to define the model by instantiating the class `LeNet` and then loading the `weights` by `LeNet.load_state_dict()`.  
However, the `.pt` file is a stand-alone file that contains the necessary information to define the model. 

##  <font color='green'>Converting from PyTorch to Libtorch</font>

In [1]:
# Let's import the necessary libraries.
import torch
import torch.nn as nn
device = "cpu" if not torch.cuda.is_available() else "cuda"

In [2]:
# create a network in Pytorch
class LeNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.feature = nn.Sequential(
        #input (28,28,1), #output (24,24,6) formula [(28-5)/1]+1=24
        nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5), 
        nn.ReLU(),
         #input (24,24,6), output (12,12,6)
        nn.MaxPool2d(kernel_size=2),
        #input (12,12,6) output (8,8,16) formula [(12-5)]+1 = 8
        nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5), 
        nn.ReLU(),
        #input (8,8,16) output (4,4,16)
        nn.MaxPool2d(kernel_size=2) 
    )
    self.classifier = nn.Sequential(
        nn.Linear(16*4*4,512), #input(4*4*16) output 512
        nn.ReLU(),

        nn.Linear(512,128),
        nn.ReLU(),
        nn.Linear(128,10)
    )
  def forward(self,x):
    x = self.feature(x)
    x = x.view(x.shape[0],-1)
    x = self.classifier(x)
    return x

Model = LeNet().to(device)

Once we have the `.pt` model, we can load this in Libtorch and infer on a random input.

In [3]:
# create an IR of the `LeNet` class and save it as `LeNet.pt`.
script_model = torch.jit.script(Model)
torch.jit.save(script_model, "./pytorch-to-libtorch/LeNet.pt")

## <font color='green'> Inferring the LeNet.pt Model in Libtorch</font>

For inferring the `.pt` model, we need to write the following C++ code.

```C++
// Load the model insde the `LeNet`
torch::jit::script::Module LeNet = torch::jit::load("./LeNet.pt");
                                        
// Create a random-input
auto input = torch::randn({3, 1, 28, 28});

// Create an input of type `torch::jit::IValue` since the model is based on `jit`.
std::vector<torch::jit::IValue> jit_input;
jit_input.push_back(input);

// Call the forward function over the input and convert it into `torch.Tensor`
auto output = LeNet.forward(jit_input).toTensor();

// Calculate the size of the output.
std::cout<<"Output size is "<<output.sizes()<<std::endl;
```

**All the code shown above has been written in a C++ file. Let's run a few bash commands to execute that code. The following code cells will build and run the code in the  CPP file.**

In [4]:
%cd pytorch-to-libtorch

/home/chetan/projects/piethon/pth_course/c3_w15_dl_pytorch/LibTorch/pytorch-to-libtorch


In [5]:
!ls

CMakeLists.txt	LeNet.pt  load_traced_LeNet.cpp


In [6]:
!mkdir build

In [7]:
%cd build

/home/chetan/projects/piethon/pth_course/c3_w15_dl_pytorch/LibTorch/pytorch-to-libtorch/build


In [8]:
! cmake ..

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Found Torch: /home/chetan/projects/piethon/pth_course/c3_w15_d

In [9]:
!cmake --build . --config Release

Scanning dependencies of target run-traced-model
[ 50%] Building CXX object CMakeFiles/run-traced-model.dir/load_traced_LeNet.cpp.o
[100%] Linking CXX executable run-traced-model
[100%] Built target run-traced-model


In [10]:
%cd ..

/home/chetan/projects/piethon/pth_course/c3_w15_dl_pytorch/LibTorch/pytorch-to-libtorch


In [11]:
!./build/run-traced-model

Successfully loaded the traced model into Libtorch .
Inferring on a random input 
Output size is [3, 10]
